In [1]:
import sklearn.model_selection
import pandas as pd
import numpy as np
import math

from hw3_traitors import *

blotter = pd.read_csv('blotter.csv')
print(blotter)

      trade_id     date asset   trip action type     price     status
0            1   1/3/20   IVV  ENTER    BUY  LMT  323.0600  SUBMITTED
1            1   1/3/20   IVV  ENTER    BUY  LMT  323.0600     FILLED
2            1   1/3/20   IVV   EXIT   SELL  LMT  326.2906  SUBMITTED
3            1   1/8/20   IVV   EXIT   SELL  LMT  326.2906     FILLED
4            2   1/6/20   IVV  ENTER    BUY  LMT  320.5700  SUBMITTED
...        ...      ...   ...    ...    ...  ...       ...        ...
2606       810  3/22/23   IVV  ENTER    BUY  LMT  398.2100  SUBMITTED
2607       810  3/22/23   IVV  ENTER    BUY  LMT  398.2100     FILLED
2608       810  3/22/23   IVV   EXIT   SELL  LMT  402.1921  SUBMITTED
2609       810  3/23/23   IVV   EXIT   SELL  LMT  402.1921       LIVE
2610       811  3/23/23   IVV  ENTER    BUY  LMT  391.5400       LIVE

[2611 rows x 8 columns]


In [2]:
ledger = blotter_to_ledger(blotter)
print(ledger)

    trade_id asset   dt_enter    dt_exit success    n       rtn
0          1   IVV 2020-01-03 2020-01-08       1    6  0.001658
1          2   IVV 2020-01-06 2020-01-08       0    3  0.000000
2          3   IVV 2020-01-07 2020-01-09       0    3  0.000000
3          4   IVV 2020-01-08 2020-01-10       0    3  0.000000
4          5   IVV 2020-01-09 2020-01-13       0    5  0.000000
..       ...   ...        ...        ...     ...  ...       ...
806      807   IVV 2023-03-17 2023-03-21       1    5  0.001990
807      808   IVV 2023-03-20 2023-03-22       0    3  0.000000
808      809   IVV 2023-03-21        NaT     NaN  NaN       NaN
809      810   IVV 2023-03-22        NaT     NaN  NaN       NaN
810      811   IVV 2023-03-23        NaT     NaN  NaN       NaN

[811 rows x 7 columns]


In [17]:
# build your set of features here.
# merge them by date to add to this dataframe.
features = pd.read_csv('daily-treasury-rates.csv')
features['Date'] = pd.to_datetime(features['Date'])

vix = pd.read_csv('^VIX.csv')[['Date', 'Open']]
vix['Date'] = pd.to_datetime(vix['Date'])

features = features.merge(vix, on='Date')

In [28]:
import numpy as np
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import StandardScaler

# Make a training set and let's try it out on two upcoming trades.
# Choose a subset of data:
X = features.drop('Date', axis=1).head(50)
x_test = features.drop('Date', axis=1).iloc[[51, 52]]
y = np.asarray(ledger.success.head(50), dtype="|S6")

print(X)
print(x_test)
print(y)

    1 Mo  2 Mo  3 Mo  4 Mo  6 Mo  1 Yr  2 Yr  3 Yr  5 Yr  7 Yr  10 Yr  20 Yr  \
0   4.34  4.50  4.80  4.91  4.92  4.59  4.08  3.87  3.67  3.62   3.57   3.91   
1   4.24  4.39  4.80  4.86  4.90  4.55  4.02  3.84  3.63  3.60   3.55   3.90   
2   4.22  4.47  4.91  4.90  4.86  4.51  3.94  3.79  3.59  3.57   3.53   3.90   
3   4.28  4.48  4.74  4.78  4.76  4.32  3.76  3.58  3.41  3.40   3.38   3.77   
4   4.26  4.48  4.73  4.84  4.80  4.38  3.76  3.57  3.39  3.39   3.38   3.78   
5   4.16  4.56  4.79  4.91  4.95  4.56  3.96  3.76  3.54  3.53   3.48   3.83   
6   4.07  4.50  4.78  4.89  4.96  4.68  4.17  3.98  3.73  3.68   3.59   3.90   
7   4.34  4.56  4.81  4.84  4.80  4.34  3.92  3.77  3.56  3.55   3.47   3.83   
8   4.31  4.51  4.52  4.79  4.71  4.26  3.81  3.68  3.44  3.45   3.39   3.76   
9   4.22  4.66  4.74  4.92  4.94  4.49  4.14  3.99  3.72  3.67   3.56   3.87   
10  4.23  4.56  4.75  4.85  4.73  4.19  3.93  3.83  3.59  3.57   3.51   3.82   
11  4.47  4.77  4.88  4.95  4.93  4.45  

In [24]:
sc = StandardScaler()

sc.fit(X)
X_std = sc.transform(X)
X_std = sc.transform(X)
x_test_std = sc.transform(x_test)

print(X_std)


[[-1.22420877 -1.5370491  -0.00932789  0.13314853 -0.16590834 -0.69680628
  -0.82006547 -0.72293995 -0.53861252 -0.55148636 -0.44340441  0.01846419
   0.1805648  -0.72947494]
 [-1.75786561 -2.38496987 -0.00932789 -0.28294063 -0.28526685 -0.84812144
  -0.98985129 -0.81515168 -0.67689558 -0.62872535 -0.53835182 -0.04747936
   0.10340036 -0.19721533]
 [-1.86459697 -1.76830022  1.01673973  0.0499307  -0.52398388 -0.99943659
  -1.21623237 -0.96883789 -0.81517864 -0.74458382 -0.63329923 -0.04747936
   0.10340036  0.51246198]
 [-1.54440287 -1.69121651 -0.56900113 -0.9486833  -1.12077646 -1.71818357
  -1.72558981 -1.61431999 -1.43745241 -1.40111521 -1.34540482 -0.90474554
  -0.89973743  0.54047653]
 [-1.65113424 -1.69121651 -0.66228001 -0.4493763  -0.88205943 -1.49121084
  -1.72558981 -1.64505724 -1.50659394 -1.4397347  -1.34540482 -0.83880199
  -0.74540854  0.27434719]
 [-2.18479107 -1.07454686 -0.10260676  0.13314853  0.01312944 -0.81029265
  -1.1596371  -1.06104962 -0.98803247 -0.8990618  -

In [25]:
ppn = Perceptron(eta0=0.1)
ppn.fit(X_std, y)

y_pred = ppn.predict(x_test_std)
print(y_pred)

[b'-1' b'-1' b'0' b'0']


In [27]:
print(ledger.iloc[[51, 52]])

   trade_id asset   dt_enter    dt_exit success  n       rtn
51       52   IVV 2020-03-18 2020-03-25       1  8  0.001244
52       53   IVV 2020-03-19 2020-03-19       1  1  0.009950
53       54   IVV 2020-03-20 2020-03-24       1  5  0.001990
54       55   IVV 2020-03-23 2020-03-24       1  2  0.004975
